In [ ]:
%matplotlib inline
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('/Users/jorge/Projects/finance')

from portfolio import Portfolio

In [ ]:
p = Portfolio()

def gray_closed_market_days(row):
    return ['color: gray' if not row['market_day'] else '' for item in row]

def color_negative_red(s):
    return ['color: %s' % 'red' if val < 0 else 'black' for val in s]

p.by_day.tail(20)[['total_value', 'day_profit', 'day_returns', 'profit', 'twrr_annualized', 'mwrr_annualized', 'returns', 'twrr', 'last_peak_twrr', 'market_day']].style.apply(gray_closed_market_days, axis=1).apply(color_negative_red, subset=['day_returns', 'day_profit']).format({
    'total_value': '{:10,.2f}'.format,
    'profit': '{:10,.2f}'.format,
    'day_profit': '{:10,.2f}'.format,
    'returns': '{:,.2%}'.format,
    'day_returns': '{:,.2%}'.format,
    'twrr': '{:,.2%}'.format,
    'mwrr': '{:,.2%}'.format,
    'twrr_annualized': '{:,.2%}'.format,
    'mwrr_annualized': '{:,.2%}'.format,
    'last_peak_twrr': '{:,.2%}'.format,
})

In [ ]:
sns.set()
sns.set_style('whitegrid')
fig, axes = plt.subplots(6, 1, figsize=(16, 50))

plt.subplot(611)
plt.ylabel('Amounts')
ylim = p.by_day['total_value'].max() + 5000 - (p.by_day['total_value'].max() % 5000)
plt.ylim(0, ylim)
plt.title('Market Value')
plt.plot(p.by_day['total_value'], '#40C040', p.by_day['capital'], '#6060B0', linewidth=1.0)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:10,.0f}'.format(y) for y in yticks])
ax.legend(['Value', 'Capital'])

plt.subplot(612)
plt.title('Profit or Loss')
ax = plt.gca()
plt.plot(p.by_day['profit'], 'CornflowerBlue', linewidth=1.0)
ax.autoscale(axis='x', tight=True)
ax.fill_between(p.by_day.index, p.by_day['profit'], color='CornflowerBlue', alpha=0.8)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:10,.0f}'.format(y) for y in yticks])
ax.legend(['Profit or Loss'])

plt.subplot(613)
plt.title('Money-, Time-Weighted, and Total Returns')
plt.ylabel('Returns')
plt.plot(p.by_day['mwrr'], '#40C040', p.by_day['twrr'], '#C09030', p.by_day['returns'], '#80A0C0', linewidth=1.0)
ylim = max(p.by_day['twrr'].max(), p.by_day['mwrr'].max()) + 0.02 - (max(p.by_day['twrr'].max(), p.by_day['mwrr'].max()) % 0.02)
plt.ylim(-0.04, ylim)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
ax.set_yticks([k / 100.0 for k in range(-4, int(ylim * 100), 2)])
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(['Money-Weighted Real Returns', 'True Time-Weighted Returns', 'Total Returns'])
    
plt.subplot(614)
plt.title('Annualized Money- and Time-Weighted Returns')
plt.ylabel('Returns')
plt.ylim(-0.05, 0.3)
plt.plot(p.by_day['mwrr_annualized'], '#40C040', p.by_day['twrr_annualized'], '#C09030', linewidth=1.0)
plt.ylim(-0.04, 0.20)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
ax.set_yticks([k / 100.0 for k in range(-4, 20, 2)])
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(['Annualized Money-Weighted Real Returns', 'Annualized True Time-Weighted Returns'])

plt.subplot(615)
plt.title('Ticker Returns')
plt.plot(p.tickers.returns, linewidth=1.0)
ax = plt.gca()
ax.autoscale(axis='x', tight=True)
yticks = ax.get_yticks()
ax.set_yticklabels(['{:3.1f}%'.format(y * 100) for y in yticks])
ax.legend(p.tickers.returns.columns)

plt.subplot(616)
plt.title('Correlations')
mask = np.zeros_like(p.tickers.correlations, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
ax = plt.gca()
cmap = sns.diverging_palette(240, 10, as_cmap=True)
sns.heatmap(p.tickers.correlations, mask=mask, cmap=cmap, vmin=-1, vmax=1, square=True,
            linewidths=.5, cbar_kws={"shrink": .8}, annot=True, ax=ax)

plt.show()

In [ ]:
p.latest()

In [ ]:
p.allocations() * 100

In [ ]:
{x: p.tickers.volatilities[x] * 100 for x in p.tickers.volatilities}